In [ ]:
import numpy as np
np.random.seed(0)

Log concave means density of the type $Cexp(-f(x))$ where $f$ is convex.  To get points according to a mixture of log-concave densities in $d$-dimensional space, do the following. Take a one dimensional log-concave density $p_1(\cdot)$ with zero mean, generate 
$N_1$ points in the line $d$ times. Denote these by $[x_1^1(1), x_1^1(2), \cdots , x_1^1(N_1)], \cdots , [x_1^d(1), \cdots , x_1^d(N_1)]$. Then generate $N_1$ points in the $d$ dimensional space as $[x^1_1(i), \cdots  , x_1^d(i)], 1 \leq i \leq N_1$. Add a common d dimensional vector $m_1$ to all of them to shift the mean. Then multiply all of them by a nonsingular square matrix $A_1$ so that the covariance is not necessarily identity. Do this for $p_2(\cdot) , \cdots , p_K(\cdot)$. The union of all these $N = \sum_iN_i$ points is a mixture of log-concave densities with mixture weights $N_i/N$. You can also use the same $p(\cdot)$, but make sure that the  $m_i$'s are distinct and well separated.

In [ ]:
Ni = [45, 55, 38, 47, 94]   #number of points from each Gaussian
k = 5                       #number of Gaussians
d = 10                      #dimension
x = []
i = 0
m = []
sigma = [1, 3, 0.5, 0.1, 2]
A = []
for n in Ni:
  m.append(np.random.randint(-50,50,d))
  x.append([])
  for o in range(d):
    x[-1].append(np.random.normal(0,sigma[i],n))
  x[-1] = np.transpose(x[-1])
  A_n = np.random.rand(d,d)
  #A_n = np.identity(d)
  while(np.linalg.det(A_n)==0):
    A_n = np.random.rand(d,d)
  A.append(A_n)
  x[-1] = np.matmul(x[-1],A_n)
  x[-1] += m[-1]

# print(x)
print(np.array(m))
#print(np.array(x, dtype = object))
print(x[1])

[[-25   7  23  12  46 -29  25 -15   6  -6]
 [-30  -1 -42  14 -16 -20  34 -13 -28  11]
 [-40 -42  38 -30  29  44   1  39   6 -41]
 [-36   5  19 -38   7 -47 -21 -17  -2 -13]
 [-27  30  36  25  13  42 -15  26  17  26]]
[[-2.89885058e+01 -5.44220725e-01 -4.12713728e+01  1.35398523e+01
  -1.52245127e+01 -2.02988117e+01  3.28754085e+01 -1.14821859e+01
  -2.63265224e+01  1.12771593e+01]
 [-2.93266509e+01 -1.28885947e+00 -4.28796625e+01  1.55474666e+01
  -1.64880715e+01 -2.09058422e+01  3.41492840e+01 -1.16271459e+01
  -2.67803645e+01  9.74187729e+00]
 [-3.00187203e+01 -1.89237753e+00 -4.09603832e+01  1.40846188e+01
  -1.70881285e+01 -2.04722646e+01  3.35477479e+01 -1.41573600e+01
  -2.89466949e+01  1.08304047e+01]
 [-2.99583666e+01  1.60140565e-01 -4.05449061e+01  1.46788668e+01
  -1.59281753e+01 -2.09435870e+01  3.42869615e+01 -1.30038090e+01
  -2.93672453e+01  1.23178367e+01]
 [-2.96823077e+01 -1.68850033e+00 -4.17615274e+01  1.39079926e+01
  -1.44293099e+01 -1.91870318e+01  3.55737585e+01 

I suggest you read the introduction of the Kannan-Slamasian-Vempala paper to see what are the issues. I propose the following: 

Generate synthetic data according to some mixture of log concave densities in high dimension.

Use a series of several random compressive sensing matrices to get  low dimensional projections of the data.

Use some standard scheme for mixtures of gaussians such as Dasgupta or variants to find centers in each case separately.

The centers should cluster around some values with some outliers because of the problems mentioned in Kannan et al. Ignore the outliers and take the means of the clusters. 

Reconstruct the means for original data by upscaling as in compressive sensing.


If you take several projections, reconstruct the estimated true means using compressive sensing ideas, trim the samples to get rid of outliers and then take their averages. Also see if there are improvements over Dasgupta's original scheme.

Note: Dasgupta's original scheme has been used for now, but it was working correctly for now.




In [ ]:
np.mean(x[1],axis = 0)

array([-30.14620687,  -1.06828365, -41.88708808,  14.13067429,
       -16.00720205, -20.0285589 ,  33.9309517 , -13.21974809,
       -28.23861464,  10.99167253])

In [ ]:



x = np.concatenate((x[:]))

In [ ]:
m = 6
num_mat = 100
phi = np.random.normal(scale = 1/m, size = (num_mat,m,d))

In [ ]:
np.shape(phi[0])
proj = np.matmul(phi,x.T)

In [ ]:
print("dimension of projected space, points")
print(np.shape(proj[0]))

dimension of projected space, points
(100, 6, 279)


Mixture of Gaussians

In [ ]:
k = 5                       #number of Gaussians
d = m                       #dimension
wmin = 0.1                 #min mixing weight
M = 246                     #data points

q = int(M*np.sqrt(d))
l = int(M*(wmin))
p = l


proj_i = 0
with open('/content/data.csv', 'w') as writefile:
    
  for proj_i in range(num_mat):
    print(proj_i)
    #print("\n")
    writefile.write("\n")
    S_1 = np.copy(proj[proj_i].T)
    for i in range(k):
      rx = np.zeros(len(S_1),dtype = int)
      close_points = []
      for x in range(len(S_1)):
        distances = np.zeros(len(S_1))
        for y in range(len(S_1)):
          distances[y] = np.linalg.norm(S_1[x]-S_1[y])
        ordered_p = np.argsort(distances)
        rx[x] = distances[ordered_p[p]]
      rx_min_pos = np.argmin(rx)
      x = rx_min_pos
      for y in range(len(S_1)):
          distances[y] = np.linalg.norm(S_1[x]-S_1[y])
      ordered_p = np.argsort(distances)
      #print(S_1[rx_min_pos])
      for i in range(len(S_1[rx_min_pos])):
        writefile.write(str(S_1[rx_min_pos][i]))
        writefile.write(", ")
      writefile.write("\n")
      S_1 = (np.delete(S_1,ordered_p[0:p],0))
      







0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
